# This Pyhton scripts reads the data.h5 saved in Kapsel and generates the .gsd data of the squirmers to mount movies in Ovito

# Author: Federico Fadda
# Date: 5/12/2018

In [219]:
import numpy as np
import gsd.hoomd
import h5py

In [220]:
#Reading the data in the data.h5 file after the simulation:
#Hoomd schema is used:

rf = "/Users/federico/mnt/ebi/kapsel3/1000_squirmers/2/2/output/data.h5"
f = h5py.File(rf, 'r')

Lx=64
Ly=128
Lz=64
Np=1000 #number of particles in the system
N1=500
N2=500
tmax=301 #number of snapshots chosen in the input file of Kapsel

CM=np.zeros(((301,Np,3)))
CMX=np.zeros(Np)
CMY=np.zeros(Np)
CMZ=np.zeros(Np)

#To mount a movie of the squirmers in Ovito the position of each particle in the box is requested!
# TO DO: reading the orientations of the particles!!!

def create_frame(i):
        s = gsd.hoomd.Snapshot()
        s.particles.N = Np
        s.configuration.step = i
        s.configuration.box = [Lx-1, Ly-1, Lz-1, 0, 0, 0]
        s.particles.types = ['$C_{2}>0$', '$C_{2}<0$']
        s.particles.typeid = np.concatenate(([0 for n in range(0,N1)], [1 for n in range(N1,Np)]))
        CM[i,:,:]=f['trajectory_data/frame_'+str(i)+'/particle/pos'].value 
        for n in range (0,Np):
            CMX[n]=CM[i,n,0]-Lx/2 #rescale in order to fit particles correctly in the box! (strangely this doesn't work for 1 squirmer)
            CMY[n]=CM[i,n,1]-Ly/2
            CMZ[n]=CM[i,n,2]-Lz/2
        s.particles.position = [[CMX[n],CMY[n],CMZ[n]] for n in range(Np)]
        return s
    
t = gsd.hoomd.open(name='squirmer_movie.gsd', mode='wb')
t.extend( (create_frame(i) for i in range(tmax)) )   
